# Numeric Table Detection


The objective is to demonstrate detection of tables in raw texxt to use targeted table extraction and lower the cost of using Textract.
Exmple for 1M pages per month containing 19% of pages with at leaqst 1 table. 

https://calculator.aws/#/estimate?id=18c806f2696f894da0c8967a1697f1120279f693


# Notebook Setup

In this step, we will import some necessary libraries that will be used throughout this notebook. 

In [ ]:
!python -m pip install -q amazon-textract-response-parser --upgrade
!python -m pip install -q amazon-textract-caller --upgrade
!python -m pip install -q amazon-textract-prettyprinter==0.0.16
!python -m pip install -q amazon-textract-textractor --upgrade
!python -m pip install -q pymupdf 
!python -m pip install -q PyPDF2

In [ ]:
#Restart the kernel
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
import boto3
import botocore
import sagemaker
import pandas as pd
from IPython.display import Image, display, JSON, HTML
from textractcaller.t_call import call_textract, Textract_Features, call_textract_expense
from textractprettyprinter.t_pretty_print import convert_table_to_list
from trp import Document
import os


# variables
data_bucket = sagemaker.Session().default_bucket()
region = boto3.session.Session().region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')

os.environ["BUCKET"] = data_bucket
os.environ["REGION"] = region
role = sagemaker.get_execution_role()

print(f"SageMaker role is: {role}\nDefault SageMaker Bucket: s3://{data_bucket}")

s3=boto3.client('s3')
textract = boto3.client('textract', region_name=region)
comprehend=boto3.client('comprehend', region_name=region)
comprehendmedical=boto3.client('comprehendmedical', region_name=region)

In [ ]:
file="./DemoTable.pdf"
file_key=f"idp/textract/demo/{os.path.basename(file)}"

!aws s3 cp {file} s3://{data_bucket}/{file_key} --only-show-errors

In [ ]:

from IPython.display import IFrame
IFrame(file, width=1500, height=600)

---
Extract unstructured data with Amazon Textract 

Amazon Textract is an ML powered OCR service that is capable of detecting and extracting text from documents. Text data in the form of WORDS and LINES can be extracted from documents using Amazon Textract `DetectDocumentText` API. Let's extract the words and lines from the demonstration document.

In [ ]:
# Call Amazon Textract
response = call_textract(input_document="s3://{data_bucket}/"+file_key)

text=""
# Print detected text
for item in response["Blocks"]:
    if item["BlockType"] == "LINE":
        text=text+"\n"+item["Text"]
print (text)

---
# Detect Tables in raw text

In this example using a threashold of 4 characters out of the 15 most used characters being digits works well to identify if the page is mostly constituted of numeric tables.



In [ ]:
from collections import Counter


def containsatable(textcontent):
    counter = Counter(textcontent)
    mostcom = counter.most_common(15)
    countdigit=0
    for i in mostcom :
        if (i[0].isdigit()):
            countdigit=countdigit+1
    return(countdigit>=4)



In [ ]:
from trp import Document
import math
doc = Document(response)
#print(doc.pages[0])
# Iterate over elements in the document
pagenum=0
pageswithtables=[]
for page in doc.pages:
    pagenum=pagenum+1
    text=''
    for line in page.lines:
        text=text+line.text
    if (containsatable(text)):
        pageswithtables.append(pagenum)

print("List of pages with Tables:")        
print (pageswithtables)
print("Percentage of pages with tables:")    
print (str(len(pageswithtables))+"/"+str(pagenum))
print (str(math.trunc(len(pageswithtables)/pagenum*100))+"%")
    


# Call Analyse Documents only on the pages with tables
First, we build a smaller PDF document containing only the pages identified as containing tables and then we run the AnalyzeDocument call on the new document.

In [ ]:
from PyPDF2 import PdfReader, PdfWriter

pdf_file_path = file
file_base_name = pdf_file_path.replace('.pdf', '')

pdf = PdfReader(pdf_file_path)

pages = pageswithtables
pdfWriter = PdfWriter()

for page_num in pages:
    pdfWriter.add_page(pdf.pages[page_num])

pdf_out = open("temp.pdf", 'wb')
pdfWriter.write(pdf_out)
pdf_out.close()


In [ ]:
!aws s3 cp temp.pdf s3://{data_bucket}/temp.pdf --only-show-errors

In [ ]:
resp = call_textract(input_document="s3://{data_bucket}/temp.pdf", features=[Textract_Features.TABLES])
tdoc = Document(resp)
dfs = list()

for page in tdoc.pages:
    for table in page.tables:
        tab_list = convert_table_to_list(trp_table=table)
        print(tab_list)
        dfs.append(pd.DataFrame(tab_list))

df1 = dfs[0]
df2 = dfs[1]